In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
import time
from get_dataset_info import *

# %load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [6]:
def estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px, alpha):
    
    err_threshold = err_threshold_px / K[0,0]
    best_inliers = None
    best_E = None
    max_inliers = 0
    epsilon = 0
    T = n_its
    n_points = x1_norm.shape[1]

    for t in trange(n_its):

        rand_mask = np.random.choice(np.size(x1_norm,1), n_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        n_inliers = np.sum(inliers)

        if n_inliers > max_inliers:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            max_inliers = n_inliers
            print(np.sum(inliers), end='\r')

            # epsilon = max_inliers / n_points
            # T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
            # print('New T:', T, 'New epsilon:', epsilon)
            # if t >= 4*T-1:
            #     print('Bailout at iteration:', t, T)
            #     break
        
    return best_E, best_inliers

In [3]:
data_set = 0
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
n_imgs = imgs.shape[0]
n_camera_pairs = n_imgs - 1
img1_init = imgs[init_pair[0]]
img2_init = imgs[init_pair[1]]

In [12]:
# Compute and save SIFT points
marg = 0.7


# SIFT points for relative orientation
for i in range(n_camera_pairs):
    print("\nCamera pair:", i+1, "/", n_camera_pairs)
    img1 = imgs[i]
    img2 = imgs[i+1]
    x1, x2, _, _, _, _, _ = cv.compute_sift_points(img1, img2, marg)
    np.save('data/RO_x1_{}_dataset_{}.npy'.format(i, data_set), x1)
    np.save('data/RO_x2_{}_dataset_{}.npy'.format(i, data_set), x2)
    

# SIFT points for camera resectioning
x1, x2, kp1, kp2, des1, des2, _ = cv.compute_sift_points(img1_init, img2_init, marg)
np.save('data/CR_x1_{}_dataset_{}.npy'.format(init_pair[1], data_set), x1)
np.save('data/CR_x2_{}_dataset_{}.npy'.format(init_pair[1], data_set), x2)

for i in range(n_imgs):

    if i != init_pair[0] and i != init_pair[1]:
        
        print("\nImage:", i+1, "/", n_imgs)
        img2 = imgs[i]
        x1, x2 = cv.compute_sift_points_sequential(kp1, des1, img2, marg)
        np.save('data/CR_x1_{}_dataset_{}.npy'.format(i, data_set), x1)
        np.save('data/CR_x2_{}_dataset_{}.npy'.format(i, data_set), x2)


Camera pair: 0 / 1
Number of matches: 44542
Number of good matches: 16518
Number of matches: 44542
Number of good matches: 16524


In [4]:
# Load SIFT points

# SIFT points for relative orientation
x1s_norm_RO = []
x2s_norm_RO = []

for i in range(n_camera_pairs):

    x1 = np.load('data/RO_x1_{}_dataset_{}.npy'.format(i, data_set))
    x2 = np.load('data/RO_x2_{}_dataset_{}.npy'.format(i, data_set))
    x1_norm = cv.dehomogenize(K_inv @ x1)
    x2_norm = cv.dehomogenize(K_inv @ x2)
    x1s_norm_RO.append(x1_norm)
    x2s_norm_RO.append(x2_norm)


# SIFT points for camera resectioning
x1s_norm_CR = []
x2s_norm_CR = []

for i in range(n_imgs):

    if i != init_pair[0]:

        x1 = np.load('data/CR_x1_{}_dataset_{}.npy'.format(i, data_set))
        x2 = np.load('data/CR_x2_{}_dataset_{}.npy'.format(i, data_set))
        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        x1s_norm_CR.append(x1_norm)
        x2s_norm_CR.append(x2_norm)

In [9]:
# Compute absolute rotations

n_its = 1000
n_samples = 8
alpha = 0.95
P1 = cv.get_canonical_camera()
abs_rots = [cv.get_camera_rotation(P1)]

for i in range(n_camera_pairs):    
    
    x1_norm = x1s_norm_RO[i]
    x2_norm = x2s_norm_RO[i]
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, pixel_threshold, alpha)
    x1_norm_inliers = x1_norm[:,inliers]
    x2_norm_inliers = x2_norm[:,inliers]

    P2_arr = cv.extract_P_from_E(E)
    X_arr = cv.get_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
    P2, _ = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)
    R2 = cv.get_camera_rotation(P2) @ abs_rots[i]
    abs_rots.append(R2)


  2%|▏         | 19/1000 [00:00<00:05, 180.95it/s]

  6%|▌         | 58/1000 [00:00<00:04, 190.33it/s]

 27%|██▋       | 271/1000 [00:01<00:04, 180.96it/s]

 60%|█████▉    | 597/1000 [00:03<00:02, 187.60it/s]

100%|██████████| 1000/1000 [00:05<00:00, 176.50it/s]


[1.62782246e+04 1.62782246e+04 3.44247125e-13]
No. valid coords for each camera pair: [6946    0 3473 3473]
Argmax(P2_arr): 0


In [16]:
alpha = 0.5
epsilon = 0.2
s = 8
np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))

270761.0

In [12]:
plt_3D = True
save = False

x = np.linspace(0,10,11)
y = np.random.rand(11)
fig = plt.figure()
plt.plot(x,y)
fig.savefig('report-images/test.png', bbox_inches='tight')
img = cv.load_image('report-images/test.png')
plt.imshow(img)